## Imports

In [1]:
import pycaret

In [3]:
import pandas as pd
import pke
import spacy
import torch
import stanza
#import spacy_stanza
import warnings
import string
from gensim.models import KeyedVectors
#import enchant    # Pour correction orthographique de synonymes


In [4]:
import numpy as np
import re
from transformers import pipeline
from transformers import AutoModel
from transformers import AutoModelForSequenceClassification
from transformers import AutoModelWithLMHead, AutoTokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
tqdm.pandas()
warnings.filterwarnings("ignore")

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [6]:
from pycaret.classification import *
from sklearn.ensemble import RandomForestClassifier

In [7]:
import sklearn
sklearn.__version__ = '0.23.2'

In [8]:
sklearn.__version__

'0.23.2'

In [5]:
spacy.__version__

'3.2.1'

## Sélection des modèles espagnols NLP

In [6]:
# Chargement pour l'utilisation de Spacy  - Français
# Utilisation de spacy pour extraire les stopwords espagnols
nlp_es = spacy.load("es_core_news_sm")
dico_spacy = {'es':nlp_es}

In [7]:
# Stopwords Français NLTK + Spacy 
stopWords = list(nlp_es.Defaults.stop_words)
stopwords_es = list(stopwords.words('spanish'))  
stopwords_es = list(set(stopwords_es + stopWords))
stopwds_lg = {'es':stopwords_es}

## Sélection des modèles Transformers

In [8]:
ckpt1 = 'Narrativa/bsc_roberta2roberta_shared-spanish-finetuned-mlsum-summarization'
ckpt2 = 'mrm8488/bert2bert_shared-spanish-finetuned-summarization'

In [9]:
tokenizer1 = AutoTokenizer.from_pretrained("Narrativa/bsc_roberta2roberta_shared-spanish-finetuned-mlsum-summarization")

In [10]:
model1 = AutoModelForSeq2SeqLM.from_pretrained("Narrativa/bsc_roberta2roberta_shared-spanish-finetuned-mlsum-summarization")

The following encoder weights were not tied to the decoder ['roberta/pooler']
The following encoder weights were not tied to the decoder ['roberta/pooler']


In [11]:
summarizer1 = pipeline('summarization', model=model1, tokenizer=tokenizer1,truncation = "only_first")

In [12]:
tokenizer2 = AutoTokenizer.from_pretrained(ckpt2)
model2 = AutoModelForSeq2SeqLM.from_pretrained(ckpt2)
summarizer2 = pipeline('summarization', model=model2, tokenizer=tokenizer2,truncation = "only_first")

The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']


In [13]:
# Plus proche du sentiment analysis ....
text_clf1 = pipeline("text-classification", model = "symanto/xlm-roberta-base-snli-mnli-anli-xnli", truncation = "only_first")   # 10 actégories, voir hugging face

# Test sur CLF1
liste_categories = ['ENTAILMENT','NEUTRAL','CONTRADICTION']

In [14]:
#classification
# Zero shot classification (permet de chosir nos propres thèmes)
nli_model = AutoModelForSequenceClassification.from_pretrained("Recognai/bert-base-spanish-wwm-cased-xnli")
zero_tokenizer = AutoTokenizer.from_pretrained("Recognai/bert-base-spanish-wwm-cased-xnli")
text_clf2 = pipeline('zero-shot-classification', model=nli_model, tokenizer=zero_tokenizer,truncation = "only_first")

# ce modèle est un zero shot classification : catégories possibles choisies (dans la presse)
candidate_labels = ['Ciencia','Política','Educación','Noticias','Salud','Tecnología','Sociedad','Deportes','Economía','Cultura','Internacional','Medio Ambiente']

In [15]:
# Sentiment Analysis
tokenizer_sentiment = AutoTokenizer.from_pretrained("finiteautomata/beto-sentiment-analysis")
model_sentiment = AutoModelForSequenceClassification.from_pretrained("finiteautomata/beto-sentiment-analysis")
sentiment1 = pipeline("text-classification", model = model_sentiment, tokenizer = tokenizer_sentiment, truncation = "only_first")
# ATTENTION CE MODELE n°2 SE DEFINIT SUR 5 niveaux
sentiment2 = pipeline("text-classification", model = 'nlptown/bert-base-multilingual-uncased-sentiment', truncation = "only_first")

## Selection Data par langues

In [16]:
data = pd.read_csv('../data/train_data_prep_v1.csv')

In [17]:
data

,pair_id,pair_lang,source_url_1,publish_date_1,source_url_2,publish_date_2,title_1,text_1,meta_description_1,meta_keywords_1,...,text_2,meta_description_2,meta_keywords_2,Geography,Entities,Time,Narrative,Overall,Style,Tone
0,1484084337_1484110209,en_en,https://www.washingtonpost.com,Wed Jan 1 00:00:00 2020,https://www.washingtonpost.com,Wed Jan 1 00:00:00 2020,Virginia man arrested in fatal DUI crash in We...,"MARTINSBURG, W.Va. — A suspected drunken drive...",Police in West Virginia say a suspected drunke...,"['Highway Fatal-DUI-West Virginia', 'Martinsbu...",...,"PORT-AU-PRINCE, Haiti — Haitian President Jove...",Haitian President Jovenel Moïse has broken wit...,"['CB-Haiti-Political Turmoil', 'Jean', 'Haiti'...",4.0,4.000000,1.000000,4.000000,4.000000,1.666667,2.000000
1,1484396422_1483924666,en_en,https://www.stlucianewsonline.com,Wed Jan 1 21:17:15 2020,https://www.thestar.com,Wed Jan 1 00:00:00 2020,Guyana: Three injured after car crashes into u...,Share This On:\n\nPin 11 Shares\n\n(NEWS ROOM ...,-,[''],...,BERLIN - A fire at a zoo in western Germany in...,BERLIN - A fire at a zoo in western Germany in...,"['smg2_world', 'smg_europe', 'smg2_news']",4.0,4.000000,1.000000,4.000000,3.666667,1.666667,1.333333
2,1484698254_1483758694,en_en,https://www.teaparty.org,NaN,https://www.timesofisrael.com,NaN,Trump Brings In 2020 At Mar-a-Lago: ‘We’re Goi...,(Breitbart) – President Donald Trump welcomed ...,NaN,[''],...,"PALM BEACH, United States — US President Donal...",US president says response to rioting by pro-I...,[''],1.0,2.000000,1.000000,2.333333,2.333333,1.000000,1.333333
3,1576314516_1576455088,en_en,https://gadgets.ndtv.com,NaN,https://gadgets.ndtv.com,NaN,Zomato Buys Uber's Food Delivery Business in I...,Uber has sold its online food-ordering busines...,Zomato on Tuesday announced it has acquired Ub...,['zomato uber eats business acquisition india ...,...,Rapid digitisation and growth in both online b...,The report by Google and Boston Consulting Gro...,['indian online food delivery market to hit us...,1.0,2.333333,2.666667,1.666667,2.000000,1.666667,1.666667
4,1484036253_1483894099,en_en,https://news.yahoo.com,Wed Jan 1 08:57:59 2020,NaN,NaN,"India approves third moon mission, months afte...",BENGALURU (Reuters) - India has approved its t...,India has approved its third lunar mission mon...,[''],...,NaN,NaN,NaN,1.0,1.250000,1.000000,1.250000,1.250000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4959,1586195445_1598778991,tr_tr,http://www.haberler.com,NaN,https://www.haberler.com,NaN,"BM, Aden'de 2 bini aşkın iç göçmenin selden za...","BM, Aden'de 2 bini aşkın iç göçmenin selden za...","Birleşmiş Milletler (BM), 2 bini aşkın iç göçm...","['Birleşmiş Milletler', 'Twitter', 'Yemen', 'G...",...,BM'den Yemen'de kadınların doğumda ölüm riski ...,BİRLEŞMİŞ Birleşmiş Milletler (BM) dünyanın en...,"['Birleşmiş Milletler', 'Yemen', 'Güncel', 'Ha...",1.0,2.000000,2.000000,4.000000,3.000000,1.000000,1.000000
4960,1590915424_1590940388,tr_tr,https://www.haberler.com,NaN,https://www.aksam.com.tr,NaN,Kovid-19'dan dolayı La Liga kulüplerinde hayat...,Kovid-19'dan dolayı La Liga kulüplerinde hayat...,Yeni tip koronavirüs (Kovid-19) salgınının eko...,"['Real Madrid', 'İspanya', 'La Liga', 'Futbol'...",...,Yeni tip koronavirüs (Kovid-19) salgınının eko...,Yeni tip koronavirüs (Kovid-19) salgınının eko...,"['İspanya 1. Futbol Ligi', 'la liga', 'koronav...",1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
4961,1526157103_1492737005,tr_tr,http://www.samanyoluhaber.com,Thu Feb 20 10:47:10 2020,https://www.fotomac.com.tr,Sun Jan 12 00:00:00 2020,Saray da çare olmadı: 'Borca boğulan dev kulüp...,\n\n\n\n\n\n\n\nİflas noktasındaki kulüplerin ...,SARAY DA ÇARE OLMADI: 'BORCA BOĞULAN DEV KULÜP...,"['Saray', 'da', 'çare', 'olmadı:', ""'Borca"", '...",...,"TFF, resmi internet sitesinden Beşiktaş'ın fai...","Federasyon, Başkan Çebi’nin yaptığı açıklamala...",[''],1.0,2.000000,3.0

In [18]:
# remémorer numéro de ligne - compléter les Nan
#data['ligne'] = data.index
data = data.fillna('')

In [19]:
# séparation des datasets
espagnol = data.loc[data.pair_lang == 'es_es',['title_1','title_2','text_1','text_2','Geography', 'Entities',
       'Time', 'Narrative', 'Overall', 'Style', 'Tone']].reset_index(drop=True)

In [20]:
espagnol.head(10)

,title_1,title_2,text_1,text_2,Geography,Entities,Time,Narrative,Overall,Style,Tone
0,Se realizan compras de última hora para fin de...,Heridos de pirotecnia,"PROGRESO, Yucatán, 31 de diciembre.- Cuando al...","Este miércoles a la madrugada, en el marco de ...",4.0,4.0,1.0,4.0,4.0,1.0,2.0
1,Realizarán obras de mejoramiento en el Bosque ...,Arica: Con ambiente festivo se inauguró la últ...,Las obras iniciarán durante el primer trimestr...,"En la Población Tarapacá Oriente, los vecinos ...",3.0,3.5,1.0,2.5,3.0,1.0,1.0
2,Estos son los países más peligrosos para viaja...,Condena FIP 49 asesinatos de periodistas en 2019,Es probable que uno de los propósitos más repe...,Los 49 asesinatos de este año ocurrieron en un...,2.0,4.0,3.0,4.0,3.0,2.0,1.0
3,Un niño sufrió quemaduras luego de poner una c...,Australia recibe el año 2020 con devastadores ...,01/01/2020 - 12:32 Policiales\n\nEn las última...,"Según los medios, más de 200 viviendas fueron ...",4.0,4.0,1.0,4.0,4.0,1.0,1.0
4,Francisco pidió «disculpas» por el incidente c...,El Papa denuncia la violencia contra las mujer...,El papa Francisco pidió hoy «disculpas» por el...,"En su primera misa del año 2020, el Papa Franc...",1.0,1.0,1.0,3.0,2.0,2.0,2.0
5,"Multan a hijo de payaso ""Chimbombín"" en Comala...",Policía multa al hijo del payaso Chimbombín po...,"El hijo de Douglas Hernández Fajardo, conocido...",El padre intercedió para que su hijo no fuera ...,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,,,,,2.0,4.0,2.0,4.0,4.0,1.0,3.0
7,Los primeros bebés de 2020 y la madre que dio ...,En Rosario las primeras bebas del 2020 fueron ...,"Martina, la hija de una mujer platense que est...",Ambas nacieron durante las primeras horas del ...,2.0,3.0,1.0,3.0,2.0,2.0,1.0
8,Para comenzar bien el año: el golazo de chilen...,"Una chilena en la Premier League, el primer go...",¿encontraste un error? avísanos\n\nEnvíanos tu...,Este año nuevo 2020 empezó con mucha actividad...,1.0,3.0,1.0,1.0,1.0,3.0,2.0
9,Respirando mina inspirado por León,Las nocheviejas infantiles triunfan,"Samuel Teijido Fraga, un gallego de 26 años af...",Las nocheviejas infantiles triunfan y se conso...,3.0,4.0,1.0,4.0,4.0,1.0,1.0


In [21]:
liste_labels = ['positive','negative','neutral']
# ATTENTION CE MODELE SE DEFINIT SUR 4 CLASSES : mixed, positif, negatif, objectif
liste_sentiments = ['1 star','2 stars','3 stars','4 stars','5 stars']
dico_classifiers = {'text_clf1': 'score_classif1','text_clf2':'score_classif2','sentiment1':'score_sentiment1',
                    'sentiment2': 'score_sentiment2'}
dico_categories = {'text_clf1': liste_categories,'text_clf2':candidate_labels,'sentiment1':liste_labels,
                    'sentiment2': liste_sentiments}

## Tests divers et Fonctions nécessaires

In [22]:
print(espagnol.title_1[5])
print(espagnol.text_1[5])
print('-----------------------')
print(espagnol.title_2[5])
print(espagnol.text_2[5])

Multan a hijo de payaso "Chimbombín" en Comalapa cuando ambos iban a "ganarse los frijolitos"
El hijo de Douglas Hernández Fajardo, conocido artísticamente como el payaso "Chimbombín", fue multado este miércoles por la mañana, tras ser detenido en un retén policial y detectarle que manejaba sin licencia de conducir el vehículo en el que se transportaba junto con su padre.

El hecho sucedió frente al monumento Cristo de La Paz, en la carretera a Comalapa. El retén se encontraba ubicado sobre el carril que conduce hacia el Aeropuerto Internacional Monseñor Romero, como parte de los planes de control policial que habitualmente se instalan esa carretera en días festivos.

Cuando fue detenido, al hijo de "Chimbombín" se le impuso una multa económica de $ 57.14, ya que no pudo presentar la licencia en vista de que solo le estaba "ayudando a manejar" a su papá en ese momento.

"Hemos saturado todas las fiestas y mi hijo me viene ayudando a manejar. Entonces, le pido de favor a ella (la agente

In [23]:
print(espagnol.title_1[3])
print(espagnol.text_1[3])
print('-------------------------------------------')
print(espagnol.title_2[3])
print(espagnol.text_2[3])

Un niño sufrió quemaduras luego de poner una cañita voladora en un bidón con nafta
01/01/2020 - 12:32 Policiales

En las últimas horas, un nene de 8 años fue internado en el Centro Provincial de Salud Infantil tras haber sufrido graves quemaduras luego de que tratara de encender una cañita voladora en un bidón de combustible.

Según trascendidos, el menor puso la pirotecnia en un bidón que hasta hace algunos momentos había contenido nafta. Las chispas de la cañita y el vapor del combustible generaron el estallido del bidón.

La explosión le causó importantes quemaduras al pequeño, principalmente en el rostro, por lo que permanece internado. Los médicos son cautelosos con el diagnóstico y esperan su evolución en las próximas horas.
-------------------------------------------
Australia recibe el año 2020 con devastadores incendios en su territorio.
Según los medios, más de 200 viviendas fueron destruidas y algunos pueblos se han convertido en ruinas humeantes. Ante la situación, las auto

In [24]:
print(summarizer1(espagnol.text_1[3]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'summary_text': 'La explosión de una cañona en la que se prendió fuego a un bidón de gasolina ha dejado al menos dos muertos'}]


In [25]:
# Résumés
print(summarizer1(espagnol.text_1[3])[0]['summary_text'])
print(summarizer1(espagnol.text_2[3])[0]['summary_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


La explosión de una cañona en la que se prendió fuego a un bidón de gasolina ha dejado al menos dos muertos
Una selección de imágenes con los momentos más impactantes del año


In [26]:
# Résumés
print(summarizer2(espagnol.text_1[3])[0]['summary_text'])
print(summarizer2(espagnol.text_2[3])[0]['summary_text'])

El menor, de 8 años, ha sido ingresado en el Hospital de Salud Infantil
Las autoridades australianas han comenzado un riguroso plan para ayudar a las miles de personas que tuvieron que celebrar año nuevo refugiados en playas


## Fonctions

In [27]:
def score_similarite(premise,hypothesis, nli_model):
    # Compute a probability P(premise|hypothesis) : Score de similarités entre 2 titres - Par encodage
    # attention pour les textes : tensor trop long, pb résolu en utilisant : truncation = "only_first"
    x = zero_tokenizer.encode(premise, hypothesis,truncation = "only_first", return_tensors='pt')
    
    logits = nli_model(x)[0]
    entail_contradiction_logits = logits[:,::2]
    probs = entail_contradiction_logits.softmax(dim=1)
    prob_label_is_true = probs[:,0]
    return prob_label_is_true[0].tolist() * 100

In [28]:
# TESTS SCORES DE SIMILARITES : Titres, Textes complets (attention Truncation ...)
print(score_similarite(espagnol.title_1[3],espagnol.title_2[3], nli_model))
print(score_similarite(espagnol.text_1[3],espagnol.text_2[3], nli_model))
print(score_similarite(summarizer1(espagnol.text_1[3])[0]['summary_text'],summarizer1(espagnol.text_2[3])[0]['summary_text'], nli_model))
print(score_similarite(summarizer2(espagnol.text_1[3])[0]['summary_text'],summarizer2(espagnol.text_2[3])[0]['summary_text'], nli_model))

45.47174274921417


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


65.23393392562866


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


14.436545968055725
49.87524151802063


In [29]:
# TESTS SCORES DE SIMILARITES : Titres, Textes complets (attention Truncation ...)
print(score_similarite(espagnol.title_1[0],espagnol.title_2[0], nli_model))
print(score_similarite(espagnol.text_1[0],espagnol.text_2[0], nli_model))
print(score_similarite(summarizer1(espagnol.text_1[0])[0]['summary_text'],summarizer1(espagnol.text_2[0])[0]['summary_text'], nli_model))
print(score_similarite(summarizer2(espagnol.text_1[0])[0]['summary_text'],summarizer2(espagnol.text_2[0])[0]['summary_text'], nli_model))

56.7547082901001


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


33.731821179389954


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


59.662652015686035
63.51804733276367


In [30]:
# Fonctions de summarization 
def summarization(texte):
    return summarizer1(texte)[0]['summary_text'], summarizer2(texte)[0]['summary_text']

In [31]:
def summaryEtComparatifs(df, i, modelezeroshot):
    resultats= pd.DataFrame(columns= ['summary1_text1', 'summary1_text2', 'score_similarite_titres', 'score_similarite_resume1', 'score_similarite_resume2'])
    
    dico_res = {}
        
    # Summary et comparatifs 
    dico_res['summary1_text1'],dico_res['summary2_text1'] = summarization(df.text_1[i])
    dico_res['summary1_text2'],dico_res['summary2_text2'] = summarization(df.text_2[i])
    dico_res['score_similarite_titres'] = score_similarite(df.title_1[i],df.title_2[i], modelezeroshot)
    dico_res['score_similarite_resume1'] = score_similarite(dico_res['summary1_text1'],dico_res['summary1_text2'], modelezeroshot)
    dico_res['score_similarite_resume2'] = score_similarite(dico_res['summary2_text1'],dico_res['summary2_text2'], modelezeroshot)
    #resultats= resultats.append(dico_res, ignore_index= True)
    return dico_res

In [32]:
# transformation des résultats du transformer type1
def transform_text_clf1(liste_dico):
    res = {}
    for dic in liste_dico:
        res[dic['label']] = dic['score']
    return res

In [33]:
# transformation des résultats du transformer type2
def transform_text_clf2(liste_cat,liste_sc):
    res = {}
    for i in range(len(liste_cat)):
        res[liste_cat[i]] = liste_sc[i]
    return res

In [34]:
# Fonctions de classification et sentiment analysis
def classification(texte,clf):
    # assume nms des claasifiers et methode de transformation
    if clf == "text_clf1":
        try:
            scores = transform_text_clf1(text_clf1(texte,return_all_scores=True)[0])
        except:
            return 'error'
        else:
            return scores
    elif clf == "text_clf2":                                 
        try:
            classes = text_clf2(texte,candidate_labels)
        except:
            return 'error'
        else:
            return transform_text_clf2(classes['labels'],classes['scores'])                          
    elif clf == "sentiment1":
        try:
            scores = transform_text_clf1(sentiment1(texte,return_all_scores=True)[0])
        except:
            return 'error'
        else:
            return scores
    elif clf == "sentiment2":
        try:
            scores = transform_text_clf1(sentiment2(texte,return_all_scores=True)[0])
        except:
            return 'error'
        else:
            return scores
    else:
        return 'error'

In [35]:
# Fonction de calcul du score (produit scalaire) pour résultats de classifaction
def fonction_produit_dotcom(liste_categor, dico_scores1,dico_scores2):
    """"dico scores sont les résultats obtenus pour chaque catégorie des textes 1 et 2"""
    result = 0.0
    for cat in liste_categor:
        result += round(dico_scores1[cat] * dico_scores2[cat],4)
    return result * 100

In [36]:
# analyse de textes classification et de sentiments
def analyseClassifEtSentiments(texte1, texte2, dico_classifiers):
    dico_res = {}
    if len(texte1)>0 and len(texte2)>0:
        for classifier in dico_classifiers.keys():
            scores1 = classification(texte1,'text_clf1')
            scores2 = classification(texte2,'text_clf1')
            if scores1 != 'error' and scores2 != 'error':
                dico_res[dico_classifiers[classifier]] = fonction_produit_dotcom(liste_categories, scores1,scores2)
            else:
                scores1 = classification(df.title_1[i],'text_clf1')
                scores2 = classification(df.title_2[i],'text_clf1')
                if scores1 != 'error' and scores2 != 'error':
                    dico_res[dico_classifiers[classifier]] = fonction_produit_dotcom(dico_categories[classifier], scores1,scores2)
                else:
                    dico_res[dico_classifiers[classifier]] = None
    return dico_res

In [37]:
# Prétraitement NLP pour PKE : suppression des mots de moins de 2 lettres non numériques
def supp_moins_2_lettres_stopwords(phrase,stopwd):
    temp = phrase.split(' ')
    res = ''
    for mot in temp:
        if mot not in stopwd and (len(mot)>2 or (len(mot)>0 and mot[0] in ['0','1','2','3','4','5','6','7','8','9'])):
            res += mot + ' '
    return res[:-1]

# Prétraitement NLP pour PKE : suppression des traits d'union(regroupe)/ des apostrophes / ponctuations
def preprocessing(texte,stopmots):
    # modifications simples des textes : ponctuations, petits mots, stopwords (à faire pour entités et pke textes)
    texte=re.sub('\'',' ',texte)   # suppression apostrophe
    texte=re.sub('-','',texte)    # suppression trait union
    regex = re.compile('[%s]' % re.escape(string.punctuation)) # suppression de toutes les ponctuations
    texte=regex.sub(' ',texte)
    texte = supp_moins_2_lettres_stopwords(texte,stopmots)
    return texte

In [38]:
def entites_communes(nlp,text1,text2, isStanza):
    """"
    Cette première fonction ne regarde que les entités communes : personnes, dates, groupe, localisations
    Elle sera appliquée aux textes et aux titres et cumulé : si cumul en titre et texte : compte double !"""
    
    doc1 = nlp(text1)
    doc2 = nlp(text2)
    nb_commun_ent = 0; liste_commun_ent = []
    nb_commun_geo = 0; liste_commun_geo = []
    nb_commun_dat = 0; liste_commun_dat = []
    
    if len(doc1.ents)>0 and len(doc2.ents)>0:
        liste1 = []; dico1 = {}
        for elt in doc1.ents:
            if isStanza:
                label= elt.type
            else:
                label= elt.label_
            if label == 'PERSON' and ' ' in elt.text:
                mots = elt.text.split(' ')
                for mot in mots:
                    if mot not in liste1:
                        liste1.append(mot)
                        dico1[mot] = label
            elif label in ['LOC','ORG','GPE','DATE','TIME']:
                if elt.text not in liste1:
                    liste1.append(elt.text)
                    
                    dico1[elt.text] = label
        liste2 = []
        for elt in doc2.ents:
            if isStanza:
                label= elt.type
            else:
                label= elt.label_
            if label == 'PERSON' and ' ' in elt.text:
                mots = elt.text.split(' ')
                for mot in mots:
                    if mot not in liste2:
                        liste2.append(mot)
            elif label in ['LOC','ORG','GPE','DATE','TIME']:
                if elt.text not in liste2:
                    liste2.append(elt.text)
        
        # points communs des listes        
        for elt in liste1:
            if elt in liste2:
                if dico1[elt] == 'LOC':
                    nb_commun_geo += 1
                    liste_commun_geo.append(elt)
                elif dico1[elt] in ['DATE','TIME']:
                    nb_commun_dat += 1
                    liste_commun_dat.append(elt)
                else:
                    nb_commun_ent += 1
                    liste_commun_ent.append(elt)
                    
    return nb_commun_ent, liste_commun_ent,nb_commun_geo, liste_commun_geo,nb_commun_dat, liste_commun_dat

In [39]:
# ENTITES COMMUNES : on tient compte des bigrammes Noms qui posent erreurs ex: Joe Biden et Biden 
# Ici, on considère mieux le CUMUl titres et Textes avec une pondération double pour le titre 
# Il faut aussi enlever les petits mots donc pré-traitement en texte
def processingEntitesCommunes(df, i,nlp_stanza, nlp_spacy):
    dico_res = {}
    nb_ent1,list_ent1,nb_geo1,list_geo1,nb_dat1,list_dat1 = entites_communes(nlp_stanza,df.title_1[i],df.title_2[i], True)
    nb_ent2,list_ent2,nb_geo2,list_geo2,nb_dat2,list_dat2 = entites_communes(nlp_stanza,df.text_1[i],df.text_2[i], True)
    if nlp_spacy != None:
        nb_ent3,list_ent3,nb_geo3,list_geo3,nb_dat3,list_dat3 = entites_communes(nlp_spacy,df.title_1[i],df.title_2[i], False)
        nb_ent4,list_ent4,nb_geo4,list_geo4,nb_dat4,list_dat4 = entites_communes(nlp_spacy,df.text_1[i],df.text_2[i], False)
    else:
        nb_ent3,list_ent3,nb_geo3,list_geo3,nb_dat3,list_dat3 = (0,[],0,[],0,[])
        nb_ent4,list_ent4,nb_geo4,list_geo4,nb_dat4,list_dat4 = (0,[],0,[],0,[])
    dico_res['nb_entites_idem'] = nb_ent1 * 2 + nb_ent2 + nb_ent3 * 2 + nb_ent4
    dico_res['nb_lieux_idem'] = nb_geo1  * 2 + nb_geo2 + nb_geo3  * 2 + nb_geo4
    dico_res['nb_dates_idem'] = nb_dat1 * 2 + nb_dat2 + nb_dat3 * 2 + nb_dat4
    # fusion des listes en supprimant les doublons
    dico_res['entites_idem'] = list(set(list_ent1+list_ent2+ list_ent3+list_ent4))
    dico_res['lieux_idem'] = list(set(list_geo1+list_geo2+list_geo3+list_geo4))
    dico_res['dates_idem'] = list(set(list_dat1+list_dat2+list_dat3+list_dat4))
    
    return dico_res

In [40]:
# PKE : Analyse des termes principaux dans les textes et titres 
# Problème 
def transformation_pke_results(res1,res2, correct_ortho = False):
    """
    Transformation des resultats de PKE : Pb bigramme peuvent ne pas être ds les 2 textes mais 1 mot seulement
    liste de clés et dictionnaires de valeurs, bigrammes jouera ainsi de maniere coefficientée 
    Exemple : fuite eau:0.05 --> 3 mots au final : fuite, eau, fuite eau : 0.05
    De plus on ajoute les synonymes issus de gensim en les coefficiant et vérifiant que cela """
    
    liste1 = []; liste2 = [] ; dico1 = {}; dico2 = {}
    for elt in res1:
        liste1.append(elt[0])
        dico1[elt[0]] = round(elt[1],3)
        if ' ' in elt[0]:    # bigramme dans ce cas, ajout des 2 mots
            liste = elt[0].split(' ')
            for mot in liste:
                liste1.append(mot)
                dico1[mot] = round(elt[1],3)
                try:
                    synonyms = ajout_synonymes(mot,correct_ortho = correct_ortho)
                except:
                    pass
                else:
                    for syn in synonyms:
                        liste1.append(syn[0])   # Ajout du mot 
                        dico1[syn[0]] = round(elt[1] * syn[1], 3)  # poids considéré
                    
    for elt in res2:
        liste2.append(elt[0])
        dico2[elt[0]] = round(elt[1],3)
        if ' ' in elt[0]:
            liste = elt[0].split(' ')
            for mot in liste:
                liste2.append(mot)
                dico2[mot] = round(elt[1],3)
                try:
                    synonyms = ajout_synonymes(mot,correct_ortho = correct_ortho)
                except:
                    pass
                else:
                    for syn in synonyms:
                        liste2.append(syn[0])   # Ajout du mot 
                        dico2[syn[0]] = round(elt[1] * syn[1], 3)  # poids considéré
    
    # similarites entre les 2 listes issus de pke avec poids
    sim = 0
    for elt in liste1:
        if elt in liste2:
            sim += (dico1[elt] + dico2[elt])/2
    return sim

In [41]:
def pkeTopicExtraction(texte1, texte2,nb_mots, test_position, langue):
    dico_res = {}
    for j,meth in enumerate(test_position):
        nom ='meth'+str(j+1)
        nb_mots_meth = nb_mots[nom]
        if len(texte1)>0 and len(texte2)>0:
            extractor = pke.unsupervised.TopicRank()
            extractor.load_document(input=texte1,language=langue,normalization="stemming")
            extractor.candidate_selection(pos=meth)
            extractor.candidate_weighting()
            keyphrases3 = extractor.get_n_best(n=nb_mots_meth)
            extractor = pke.unsupervised.TopicRank()
            extractor.load_document(input=texte2,language=langue,normalization="stemming")
            extractor.candidate_selection(pos=meth)
            extractor.candidate_weighting()
            keyphrases4 = extractor.get_n_best(n=nb_mots_meth)
            dico_res[nom+'_similarites'] = round(100*transformation_pke_results(keyphrases3,keyphrases4),1)
        else:
            dico_res[nom+'_similarites'] = 'Error'
    return dico_res

In [42]:
# Choix des paramètres de la méthode : A revoir ?
methode1 = {"NOUN", "PROPN", "ADJ","VERB"}
methode2 = {"NOUN", "PROPN", "ADJ"}
nb_mots = {'meth1': 30, 'meth2':50}
def Creation_features_comparaison(df,langue,dico_spacy, modelezeroshot, stopwds_lg, dico_classifiers,nb_mots, test_position = [methode1,methode2]):
    """Création des notes pour classification ensuite"""
    
    resultats = pd.DataFrame(columns = ['summary1_text1','summary2_text1','summary1_text2','summary2_text2',
            'nb_entites_idem','nb_lieux_idem', 'nb_dates_idem','entites_idem','lieux_idem','dates_idem',
            'score_similarite_titres','score_similarite_resume1','score_similarite_resume2'])
    
    # initialisation de la langue stanza
    stanza.download(langue)
    # Initialize the pipeline
    #nlp_stanza = spacy_stanza.load_pipeline(langue)
    nlp_stanza = stanza.Pipeline(lang=langue, processors='tokenize,ner')
    
    if langue in dico_spacy.keys():
        nlp_spacy = dico_spacy[langue]
    else:
        nlp_spacy = None
    
    stopmts = stopwds_lg[langue]
    for i in tqdm(range(len(df))):
        
        # Summary et comparatifs 
        resultatsSummary = summaryEtComparatifs(df,i, modelezeroshot)
        
        # analyse de textes classification et de sentiments
        texte1 = df.title_1[i] + ' ' + df.text_1[i]
        texte2 = df.title_2[i] + ' ' + df.text_2[i]
        resultatsClassif = analyseClassifEtSentiments(texte1, texte2, dico_classifiers)
        
        # pré traitement des textes pour entités et PKE
        texte1 = preprocessing(texte1, stopmts)
        texte2 = preprocessing(texte2, stopmts)
        
        # Entités communes
        resultatsEntitesCommunes = processingEntitesCommunes(df, i, nlp_stanza, nlp_spacy)
        
        #PKE : topic rank
        resultatspkeTopicExtract= pkeTopicExtraction(texte1, texte2, nb_mots, test_position, langue)
        
        resultats= resultats.append([resultatsSummary, resultatsClassif, resultatsEntitesCommunes, resultatspkeTopicExtract], ignore_index= True)
    return resultats

In [43]:
similarites = Creation_features_comparaison(espagnol.reset_index(drop=True),"es", dico_spacy, nli_model, stopwds_lg, dico_classifiers, nb_mots)

2021-12-28 12:58:32 INFO: Downloading default packages for language: es (Spanish)...
2021-12-28 12:58:34 INFO: File exists: C:\Users\camille.gosset\stanza_resources\es\default.zip.
2021-12-28 12:58:38 INFO: Finished downloading models and saved to C:\Users\camille.gosset\stanza_resources.
2021-12-28 12:58:38 WARNING: Language es package default expects mwt, which has been added
2021-12-28 12:58:38 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| ner       | conll02 |

2021-12-28 12:58:38 INFO: Use device: cpu
2021-12-28 12:58:38 INFO: Loading: tokenize
2021-12-28 12:58:38 INFO: Loading: mwt
2021-12-28 12:58:38 INFO: Loading: ner
2021-12-28 12:58:39 INFO: Done loading processors!


  0%|          | 0/570 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Your max_length is set to 64, but you input_length is only 2. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Setting `pad_token_id` to `eos_tok

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Your max_length is set to 64, but you input_length is only 2. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Your max_length is set to 64, but you input_length is only 2. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Your max_length is set to 64, but you input_length is only 2. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Your max_length is set to 64, but you input_length is only 2. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Setting `pad_token_id` to `eos_token_id`:2 for open-end gene

Your max_length is set to 64, but you input_length is only 2. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Your max_length is set to 64, but you input_length is only 2. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Your max_length is set to 64, but you input_length is only 2. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Your max_length is set to 64, but you input_length is only 2. You might co

...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Your max_length is set to 64, but you input_length is only 2. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Your max_length is set to 64, but you input_length is only 2. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_i

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Your max_length is set to 64, but you input_length is only 36. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=18)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Your max_length is set to 64, but you input_length is only 38. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=19)
Setting `pad_tok

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Your max_length is set to 64, but you input_length is only 2. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Your max_length is set to 64, but you input_length is only 2. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Your max_length is set to 64, but you input_length is only 2. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Your max_length is set to 64, but you input_length is only 2. You might co

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Your max_length is set to 64, but you input_length is only 2. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Your max_length is set to 64, but you input_length is only 2. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_i

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

In [44]:
predicteurs1 = ['score_similarite_titres','score_similarite_resume1','score_similarite_resume2','score_classif1','score_classif2',
            'score_sentiment1','score_sentiment2']

In [45]:
df_es = espagnol.round({'Geography':0, 'Entities':0,'Time':0, 'Narrative':0, 'Overall':0, 'Style':0, 'Tone':0})

In [46]:
partiel = df_es[['Geography', 'Entities','Time', 'Narrative', 'Overall', 'Style', 'Tone']].astype('int32')

In [47]:
partiel

,Geography,Entities,Time,Narrative,Overall,Style,Tone
0,4,4,1,4,4,1,2
1,3,4,1,2,3,1,1
2,2,4,3,4,3,2,1
3,4,4,1,4,4,1,1
4,1,1,1,3,2,2,2
...,...,...,...,...,...,...,...
565,1,2,1,2,3,2,4
566,1,2,1,2,2,1,1
567,2,1,1,2,1,1,1
568,1,2,3,4,3,1,3


In [48]:
es_final = pd.concat([espagnol,
        partiel,similarites],axis=1)

In [49]:
es_final

,title_1,title_2,text_1,text_2,Geography,Entities,Time,Narrative,Overall,Style,...,dates_idem,score_similarite_titres,score_similarite_resume1,score_similarite_resume2,score_classif1,score_classif2,score_sentiment1,score_sentiment2,meth1_similarites,meth2_similarites
0,Se realizan compras de última hora para fin de...,Heridos de pirotecnia,"PROGRESO, Yucatán, 31 de diciembre.- Cuando al...","Este miércoles a la madrugada, en el marco de ...",4.0,4.0,1.0,4.0,4.0,1.0,...,NaN,56.754708,59.662652,63.518047,NaN,NaN,NaN,NaN,NaN,NaN
1,Realizarán obras de mejoramiento en el Bosque ...,Arica: Con ambiente festivo se inauguró la últ...,Las obras iniciarán durante el primer trimestr...,"En la Población Tarapacá Oriente, los vecinos ...",3.0,3.5,1.0,2.5,3.0,1.0,...,NaN,NaN,NaN,NaN,8.41,8.41,8.41,8.41,NaN,NaN
2,Estos son los países más peligrosos para viaja...,Condena FIP 49 asesinatos de periodistas en 2019,Es probable que uno de los propósitos más repe...,Los 49 asesinatos de este año ocurrieron en un...,2.0,4.0,3.0,4.0,3.0,2.0,...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Un niño sufrió quemaduras luego de poner una c...,Australia recibe el año 2020 con devastadores ...,01/01/2020 - 12:32 Policiales\n\nEn las última...,"Según los medios, más de 200 viviendas fueron ...",4.0,4.0,1.0,4.0,4.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
4,Francisco pidió «disculpas» por el incidente c...,El Papa denuncia la violencia contra las mujer...,El papa Francisco pidió hoy «disculpas» por el...,"En su primera misa del año 2020, el Papa Franc...",1.0,1.0,1.0,3.0,2.0,2.0,...,NaN,51.384956,58.752096,38.829806,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2275,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50,22.8
2276,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,17.165859,30.353743,29.052958,NaN,NaN,NaN,NaN,NaN,NaN
2277,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,40.00,40.00,40.00,40.00,NaN,NaN
2278,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
es_final.to_csv('../data/corpus_es_notesv2.csv')

In [9]:
es_final= pd.read_csv('../data/corpus_es_notes2.csv')

In [11]:
espa = pd.concat([es_final[[ 'title_1', 'title_2', 'text_1', 'text_2','summary1_text1', 'summary2_text1', 'summary1_text2', 'summary2_text2']],
        es_final[['Geography', 'Entities','Time', 'Narrative', 'Overall', 'Style', 'Tone']],es_final[['nb_entites_idem', 'nb_lieux_idem', 'nb_dates_idem', 'entites_idem','dates_idem', 'score_similarite_titres',
       'score_similarite_resume1', 'score_similarite_resume2','score_classif1', 'score_classif2', 'score_sentiment1',
       'score_sentiment2']]],axis=1)

In [12]:
espa

,title_1,title_2,text_1,text_2,summary1_text1,summary2_text1,summary1_text2,summary2_text2,Geography,Entities,Time,Narrative,Overall,Style,Tone,nb_entites_idem,nb_lieux_idem,nb_dates_idem,entites_idem,dates_idem,score_similarite_titres,score_similarite_resume1,score_similarite_resume2,score_classif1,score_classif2,score_sentiment1,score_sentiment2
0,Se realizan compras de última hora para fin de...,Heridos de pirotecnia,"PROGRESO, Yucatán, 31 de diciembre.- Cuando al...","Este miércoles a la madrugada, en el marco de ...",El centro de la ciudad celebra el Año Nuevo en...,Los comerciantes de la zona del Malecón acudie...,"Este miércoles a la madrugada, en el marco de ...",Las consultas por quemaduras de pirotecnia se ...,4.0,4.0,1.0,4.0,4.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,56.754708,59.662652,63.518047,NaN,NaN,NaN,NaN
1,Realizarán obras de mejoramiento en el Bosque ...,Arica: Con ambiente festivo se inauguró la últ...,Las obras iniciarán durante el primer trimestr...,"En la Población Tarapacá Oriente, los vecinos ...",NaN,NaN,NaN,NaN,3.0,3.5,1.0,2.5,3.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.41,8.41,8.41,8.41
2,Estos son los países más peligrosos para viaja...,Condena FIP 49 asesinatos de periodistas en 2019,Es probable que uno de los propósitos más repe...,Los 49 asesinatos de este año ocurrieron en un...,NaN,NaN,NaN,NaN,2.0,4.0,3.0,4.0,3.0,2.0,1.0,0.0,0.0,0.0,[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Un niño sufrió quemaduras luego de poner una c...,Australia recibe el año 2020 con devastadores ...,01/01/2020 - 12:32 Policiales\n\nEn las última...,"Según los medios, más de 200 viviendas fueron ...",Las obras comenzarán durante el primer trimest...,Las obras comenzarán durante el primer trimest...,Los habitantes de la Amazonia Oriente y Occide...,"En la población Tarapacá Oriente, los vecinos ...",4.0,4.0,1.0,4.0,4.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,51.384956,58.752096,38.829806,NaN,NaN,NaN,NaN
4,Francisco pidió «disculpas» por el incidente c...,El Papa denuncia la violencia contra las mujer...,El papa Francisco pidió hoy «disculpas» por el...,"En su primera misa del año 2020, el Papa Franc...",NaN,NaN,NaN,NaN,1.0,1.0,1.0,3.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.53,2.53,2.53,2.53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.98,8.98,8.98,8.98
1706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,0.0,['TV'],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
1707,NaN,NaN,NaN,NaN,El presidente Iván Duque realiza un Facebook L...,El programa de Duque posee un espacio todos lo...,Este 27 de abril las personas ya podrán salir ...,El presidente de Estados Unidos asegura que la...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.165859,30.353743,29.052958,NaN,NaN,NaN,NaN
1708,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.00,40.00,40.00,40.00


## Z

In [82]:
es_final[predicteurs1 + ['Overall']].values

array([[56.75470829, 59.66265202, 63.51804733, ...,         nan,
         4.        ,  4.        ],
       [        nan,         nan,         nan, ...,  8.41      ,
         3.        ,  3.        ],
       [        nan,         nan,         nan, ...,         nan,
         3.        ,  3.        ],
       ...,
       [17.16585904, 30.35374284, 29.05295789, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ..., 40.        ,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan]])

In [15]:
es_classif = setup(data = espa[predicteurs1 + ['Overall']],  target = 'Overall', html=False, 
                   silent=True, verbose=False, imputation_type='iterative')

In [16]:
lr = create_model('lr')
rf = create_model('rf')
xgb = create_model('xgboost')

ValueError: Supported target types are: ('binary', 'multiclass'). Got 'continuous' instead.

In [17]:
essai_classif = es_final[['Geography','Entities', 'Time', 'Narrative', 'Overall', 'Style', 'Tone','nb_entites_idem', 
    'nb_lieux_idem', 'nb_dates_idem', 'score_similarite_titres', 'score_similarite_resume1','score_similarite_resume2', 
    'score_classif1', 'score_classif2','score_sentiment1', 'score_sentiment2', 'meth1_similarites','meth2_similarites']]

KeyError: "['meth1_similarites', 'meth2_similarites'] not in index"